<a href="https://colab.research.google.com/github/Greeshu-09/AISECT-AIML/blob/main/Cr_Valid_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:


#importing the Libraies
import numpy as np
import pandas as pd


In [3]:

# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:

dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)


In [6]:

X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [8]:
from sklearn.linear_model import LogisticRegression

In [12]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
# Small parameter grid for speed
param_grid = {
    'penalty': ['l2'],             # simple penalty
    'C': [0.1, 1, 10],             # small grid for regularization
    'solver': ['liblinear'],       # fast solver for small datasets
    'max_iter': [100]              # enough to converge
}

# GridSearchCV setup
grid = GridSearchCV(
    estimator=LogisticRegression(random_state=42),
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,            # 3-fold CV for faster computation
    refit=True,
    verbose=2,
    n_jobs=-1
)

# Fit the model
grid.fit(X, y)
# Best parameters & score
print("Best Parameters:", grid.best_params_)
print("Best f1_weighted Score:", grid.best_score_)

# Best estimator for predictions
best_logreg = grid.best_estimator_

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best Parameters: {'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best f1_weighted Score: 0.8055394074455524


In [13]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=3)

In [14]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')


In [17]:
# Save Best Model
import pickle
filename = "LogisticRegressionClassifier.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))


In [18]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_max_iter,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004238,0.000213,0.006745,0.002101,0.1,100,l2,liblinear,"{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', '...",0.745698,0.923516,0.747404,0.805539,0.083425,1
1,0.004573,0.001072,0.004887,0.000139,1.0,100,l2,liblinear,"{'C': 1, 'max_iter': 100, 'penalty': 'l2', 'so...",0.765958,0.923516,0.713672,0.801049,0.089189,2
2,0.003539,0.000193,0.004887,0.000297,10.0,100,l2,liblinear,"{'C': 10, 'max_iter': 100, 'penalty': 'l2', 's...",0.765958,0.923516,0.705474,0.798316,0.091909,3


In [19]:
# Get user input
age = int(input("Enter Age: "))
salary = float(input("Enter Estimated Salary: "))
gender_male = int(input("Enter Gender (0 for Female, 1 for Male): "))

# Preprocess input
# The order of features should match the training data: 'Age', 'EstimatedSalary', 'Gender_Male'
user_input = np.array([[age, salary, gender_male]])
user_input_scaled = sc.transform(user_input)

# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))

# Make prediction
prediction = loaded_model.predict(user_input_scaled)

# Display the prediction
if prediction[0] == 1:
    print("Likely to purchase.")
else:
    print("Unlikely to purchase.")

Enter Age: 51
Enter Estimated Salary: 23000
Enter Gender (0 for Female, 1 for Male): 1
Likely to purchase.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
